<a href="https://colab.research.google.com/github/midnightripper/testing/blob/main/Variational_Auto_Encoder2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Prerequisite

In [31]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Reshape
from tensorflow.keras import backend as K

Load the train and test data splits

In [33]:
GER_test = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_test.mat')['GER_test'])).transpose()
GER_train = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_train.mat')['GER_train'])).transpose()
test_frame = pd.DataFrame(GER_test).sort_values(1)
train_frame = pd.DataFrame(GER_train).sort_values(1)

In [34]:
print(f"x_train shape: {GER_train.shape} - y_train shape: {GER_train.shape}")
print(f"x_test shape: {GER_test.shape} - y_test shape: {GER_test.shape}")

x_train shape: (6981, 21) - y_train shape: (6981, 21)
x_test shape: (6248, 21) - y_test shape: (6248, 21)


In [35]:
def get_data_labels(split):
    x = pd.DataFrame(split)
    labels = x[0].values.astype(np.uint8)
    del x[0],x[1]
    data = x.values 
    return data, labels
    
x_train, y_train = get_data_labels(train_frame.values.tolist())
x_test, y_test = get_data_labels(test_frame.values.tolist())

In [36]:
df = pd.DataFrame(x_train)
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [37]:
df2 = pd.DataFrame(x_test)
scaler = StandardScaler()
x_test = pd.DataFrame(scaler.fit_transform(df2), columns=df.columns)

In [38]:
input_shape = (19,)

In [99]:
# Define the sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], 19), mean=0., stddev=0.004)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

# def sampling(args):
#     """Reparameterization trick. Instead of sampling from Q(z|X), 
#     sample eps = N(0,I) z = z_mean + sqrt(var)*eps.

#     Parameters:
#     -----------
#     args: list of Tensors
#         Mean and log of variance of Q(z|X)

#     Returns
#     -------
#     z: Tensor
#         Sampled latent vector
#     """

#     z_mean, z_log_var = args
#     eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=1.0, name='epsilon')
#     z = z_mean + tf.exp(z_log_var / 2) * eps
#     return z

In [100]:
inputs = keras.layers.Input(shape=input_shape, name='input')
x = keras.layers.Dense(64, activation='relu')(inputs)
z_mean = keras.layers.Dense(19, name='z_mean')(x)
z_log_var = keras.layers.Dense(19, name='z_log_var')(x)

In [102]:
z = keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])

encoder = keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')


In [42]:
# z = keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])
# encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
# encoder.summary()

In [103]:
latent_inputs = keras.layers.Input(shape=(19), name='z_sampling')
middle=keras.layers.Dense(64, activation='relu')(latent_inputs)
x = keras.layers.Dense(19, activation='relu')(middle)

# instantiate decoder model
decoder = keras.Model(latent_inputs, x, name='decoder')


In [104]:
outputs = decoder(encoder(inputs)[2])  # Select the Z value from outputs of the encoder
vae = keras.Model(inputs, outputs, name='vae')
vae.summary()

Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 19)]              0         
                                                                 
 encoder (Functional)        [(None, 19),              3750      
                              (None, 19),                        
                              (None, 19)]                        
                                                                 
 decoder (Functional)        (None, 19)                2515      
                                                                 
Total params: 6,265
Trainable params: 6,265
Non-trainable params: 0
_________________________________________________________________


In [105]:
autoencoder = Model(inputs, outputs)

In [106]:
reconstruction_loss = K.mean(K.square(outputs - inputs), axis=-1)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)


In [107]:
autoencoder.add_loss(vae_loss)
autoencoder.summary()

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 encoder (Functional)           [(None, 19),         3750        ['input[0][0]']                  
                                 (None, 19),                                                      
                                 (None, 19)]                                                      
                                                                                                  
 decoder (Functional)           (None, 19)           2515        ['encoder[0][2]']                
                                                                                           

In [108]:
# reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
# reconstruction_loss = reconstruction_loss * 19

# # KL Divergence loss
# kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
# kl_loss = -0.5 * tf.reduce_sum(kl_loss, axis=-1)
# vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

# autoencoder.add_loss(vae_loss)
autoencoder.compile(optimizer='adam')
# autoencoder.summary()

In [109]:
autoencoder.fit(x_train, epochs=50, batch_size=32, validation_data=(x_test, None))

Epoch 1/50
219/219 [==============================] - 3s 6ms/step - loss: 1.6157 - val_loss: 0.9252
Epoch 2/50
219/219 [==============================] - 1s 5ms/step - loss: 0.7943 - val_loss: 0.7040
Epoch 3/50
219/219 [==============================] - 1s 5ms/step - loss: 0.6435 - val_loss: 0.5960
Epoch 4/50
219/219 [==============================] - 1s 7ms/step - loss: 0.5683 - val_loss: 0.5502
Epoch 5/50
219/219 [==============================] - 2s 8ms/step - loss: 0.5316 - val_loss: 0.5186
Epoch 6/50
219/219 [==============================] - 1s 7ms/step - loss: 0.5086 - val_loss: 0.4981
Epoch 7/50
219/219 [==============================] - 1s 5ms/step - loss: 0.4912 - val_loss: 0.4882
Epoch 8/50
219/219 [==============================] - 1s 5ms/step - loss: 0.4805 - val_loss: 0.4765
Epoch 9/50
219/219 [==============================] - 1s 5ms/step - loss: 0.4717 - val_loss: 0.4692
Epoch 10/50
219/219 [==============================] - 1s 5ms/step - loss: 0.4650 - val_loss: 0.4614

In [111]:
encoder = Model(inputs, z)
encoder.trainable = False
encoder.summary()

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense_79 (Dense)               (None, 64)           1280        ['input[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 19)           1235        ['dense_79[0][0]']               
                                                                                                  
 z_log_var (Dense)              (None, 19)           1235        ['dense_79[0][0]']               
                                                                                           

In [112]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [124]:
classifier_input = Input(shape=(19,))
encoded_input = encoder(classifier_input)
# x1=Dense(128,activation='relu')(encoded_input)
# features = layers.Dropout(0.3)(x1)
# x2=Dense(64,activation='relu')(features)
# features = layers.Dropout(0.25)(x2)
x3=Dense(32,activation='relu')(encoded_input)
x4=Dense(16,activation='relu')(x3)
x5=Dense(8,activation='relu')(x4)
x6=Dense(4,activation='relu')(x5)
output_layer = Dense(1, activation='sigmoid')(x6)

classifier = Model(classifier_input, output_layer)
classifier.summary()
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test),callbacks=[es])


Model: "model_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 19)]              0         
                                                                 
 model_27 (Functional)       (None, 19)                3750      
                                                                 
 dense_103 (Dense)           (None, 32)                640       
                                                                 
 dense_104 (Dense)           (None, 16)                528       
                                                                 
 dense_105 (Dense)           (None, 8)                 136       
                                                                 
 dense_106 (Dense)           (None, 4)                 36        
                                                                 
 dense_107 (Dense)           (None, 1)                 5  

In [125]:
accuracy = classifier.evaluate(x_train, y_train)[1]
print(f"Train accuracy: {round(accuracy * 100, 2)}%")

219/219 [==============================] - 1s 2ms/step - loss: 0.3908 - accuracy: 0.8262
Train accuracy: 82.62%


In [126]:
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

196/196 [==============================] - 0s 2ms/step - loss: 0.4332 - accuracy: 0.8083
Test accuracy: 80.83%
